In [1]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import pickle
import itertools
from pathlib import Path

import numpy as np
import torch
# from torchsummary import summary
from models import Generator, Discriminator

In [3]:
g = Generator(512, 0, 256, extra_channels=1, is_training=True)# .to('cuda')

In [4]:
d = Discriminator(0, 256, extra_channels=1) #.to(self.device)

In [5]:
g_ema = Generator(512, 0, 256, extra_channels=1, is_training=False)

In [6]:
ckpt_path = Path("~/tmp/deepfashion_finetune/checkpoints/ckpt-105000.pt").expanduser()

In [7]:
# print('load model:', config.TRAIN.CKPT)
ckpt = torch.load(ckpt_path)

try:
    start_iter = int(ckpt_path.stem.split('-')[1])
except ValueError:
    print('**** load ckpt failed. start from scratch ****')
    pass
start_iter

105000

In [11]:
[x for x in ckpt['g'].keys() if 'trgb' in x]

['synthesis_network.trgbs.0.bias',
 'synthesis_network.trgbs.0.upsample.kernel',
 'synthesis_network.trgbs.0.conv.w',
 'synthesis_network.trgbs.0.conv.dense.w',
 'synthesis_network.trgbs.0.conv.dense.b',
 'synthesis_network.trgbs.1.bias',
 'synthesis_network.trgbs.1.upsample.kernel',
 'synthesis_network.trgbs.1.conv.w',
 'synthesis_network.trgbs.1.conv.dense.w',
 'synthesis_network.trgbs.1.conv.dense.b',
 'synthesis_network.trgbs.2.bias',
 'synthesis_network.trgbs.2.upsample.kernel',
 'synthesis_network.trgbs.2.conv.w',
 'synthesis_network.trgbs.2.conv.dense.w',
 'synthesis_network.trgbs.2.conv.dense.b',
 'synthesis_network.trgbs.3.bias',
 'synthesis_network.trgbs.3.upsample.kernel',
 'synthesis_network.trgbs.3.conv.w',
 'synthesis_network.trgbs.3.conv.dense.w',
 'synthesis_network.trgbs.3.conv.dense.b',
 'synthesis_network.trgbs.4.bias',
 'synthesis_network.trgbs.4.upsample.kernel',
 'synthesis_network.trgbs.4.conv.w',
 'synthesis_network.trgbs.4.conv.dense.w',
 'synthesis_network.trg

In [13]:
g_dict = {k: v for k,v in g.named_parameters()}

In [20]:
# generator
for k,v in g.named_parameters():
    if 'trgb' in k:
        if 'conv.w' in k:
            with torch.no_grad():
                v[:3, ...].copy_(ckpt['g'][k])
        elif 'bias' in k:
            with torch.no_grad():
                v[:, :3, ...].copy_(ckpt['g'][k])
        else:
            with torch.no_grad():
                v.copy_(ckpt['g'][k])
            
    else:
        with torch.no_grad():
            v.copy_(ckpt['g'][k])
        # fix other weights
        v.requires_grad = False 

In [25]:
# descriminator
for k,v in d.named_parameters():
    if 'frgb' in k:
        if 'conv.w' in k:
            with torch.no_grad():
                v[:, :3, ...].copy_(ckpt['d'][k])
        else:
            with torch.no_grad():
                v.copy_(ckpt['d'][k])
    else:
        with torch.no_grad():
            v.copy_(ckpt['d'][k])
        # fix other weights
        v.requires_grad = False 

In [29]:
v.requires_grad

True